In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [3]:
train_dataset = pd.read_csv(r"/content/sample_data/train_data.txt",delimiter=':::',names=['ID','TITLE','GENRE','DESCRIPTION'],engine='python')
test_dataset = pd.read_csv(r"/content/sample_data/test_data.txt",delimiter=':::',names=['ID','TITLE','DESCRIPTION'],engine='python')
#test_dataset.head(10)
print(test_dataset.shape)
train_dataset.head(10)

(54200, 3)


,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...
5,6,Quality Control (2011),documentary,Quality Control consists of a series of 16mm ...
6,7,"""Pink Slip"" (2009)",comedy,In tough economic times Max and Joey have all...
7,8,One Step Away (1985),crime,Ron Petrie (Keanu Reeves) is a troubled teen ...
8,9,"""Desperate Hours"" (2016)",reality-tv,"A sudden calamitous event, causing great loss..."
9,10,Spirits (2014/I),horror,Four high school students embark on a terrify...


In [4]:
train_dataset['DESCRIPTION'],test_dataset['DESCRIPTION'] = train_dataset['DESCRIPTION'].astype(str).apply(lambda sen:sen.lower()),test_dataset['DESCRIPTION'].astype(str).apply(lambda sen:sen.lower())
train_dataset.head(10)

,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,listening in to a conversation between his do...
1,2,Cupid (1997),thriller,a brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,as the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,to help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,the film's title refers not only to the un-re...
5,6,Quality Control (2011),documentary,quality control consists of a series of 16mm ...
6,7,"""Pink Slip"" (2009)",comedy,in tough economic times max and joey have all...
7,8,One Step Away (1985),crime,ron petrie (keanu reeves) is a troubled teen ...
8,9,"""Desperate Hours"" (2016)",reality-tv,"a sudden calamitous event, causing great loss..."
9,10,Spirits (2014/I),horror,four high school students embark on a terrify...


In [5]:
genre_list = list(set([genre for genre in train_dataset['GENRE']]))
print(genre_list)

[' adventure ', ' history ', ' action ', ' animation ', ' fantasy ', ' drama ', ' romance ', ' short ', ' musical ', ' sport ', ' music ', ' news ', ' reality-tv ', ' thriller ', ' western ', ' adult ', ' horror ', ' documentary ', ' mystery ', ' sci-fi ', ' family ', ' crime ', ' game-show ', ' comedy ', ' talk-show ', ' biography ', ' war ']


In [6]:
temp = dict()
for i in genre_list:
    temp[i] = train_dataset[train_dataset["GENRE"] == i]

In [7]:
num_dict = dict()
for i,df in enumerate(temp):
    num_dict[i] = df
print(num_dict)

{0: ' adventure ', 1: ' history ', 2: ' action ', 3: ' animation ', 4: ' fantasy ', 5: ' drama ', 6: ' romance ', 7: ' short ', 8: ' musical ', 9: ' sport ', 10: ' music ', 11: ' news ', 12: ' reality-tv ', 13: ' thriller ', 14: ' western ', 15: ' adult ', 16: ' horror ', 17: ' documentary ', 18: ' mystery ', 19: ' sci-fi ', 20: ' family ', 21: ' crime ', 22: ' game-show ', 23: ' comedy ', 24: ' talk-show ', 25: ' biography ', 26: ' war '}


In [8]:
for i,df in enumerate(temp):
    temp[df]['GENRE'] = i

<ipython-input-8-9b481f96e85b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[df]['GENRE'] = i


In [9]:
pp_train_dataset = pd.concat([temp[val] for val in temp])
pp_train_dataset

,ID,TITLE,GENRE,DESCRIPTION
44,45,The Wish Kin (????),0,fourteen-year-old colm bell and his 11-year-o...
119,120,Minas-Texas (1989),0,in a country town in the state of minas gerai...
209,210,Night Traveler (2013),0,night traveler: prophecy of the elder follows...
342,343,Wasteland Wars (2017),0,"guide our hero, brimstone - in an epic quest ..."
365,366,"""Lost in the Game"" (2016)",0,a group of teenagers receive mysterious invit...
...,...,...,...,...
52002,52003,Marcia o crepa (1962),26,a french foreign legion commander is told to ...
53040,53041,Ora X - pattuglia suicida (1969),26,when the germans are allowed to recapture an ...
53491,53492,Zvezde su oci ratnika (1972),26,"when a warrior dies, a star is born, a compan..."
53842,53843,"""Voennaya razvedka. Pervyy udar"" (2011)",26,"series ""war intelligence service: the first h..."


In [22]:
vectorizer = CountVectorizer(max_features=5000)
x_train = vectorizer.fit_transform(pp_train_dataset['DESCRIPTION'])
y_train = np.array(pp_train_dataset['GENRE'])

  (0, 256)	1
  (0, 2162)	1
  (0, 4535)	1
  (0, 4465)	1
  (0, 3129)	1
  (0, 4986)	1
  (0, 2287)	2
  (0, 660)	1
  (0, 461)	1
  (0, 3764)	1
  (0, 4462)	1
  (0, 4629)	1
  (0, 1720)	1
  (0, 591)	1
  (0, 1980)	1
  (0, 4863)	1
  (0, 4635)	1
  (0, 3353)	1
  (0, 1067)	1
  (0, 1945)	1
  (0, 3620)	1
  (0, 533)	4
  (0, 333)	1
  (1, 2162)	3
  (1, 4535)	1
  :	:
  (10842, 1826)	1
  (10842, 2287)	1
  (10842, 518)	2
  (10842, 1058)	1
  (10842, 4931)	1
  (10842, 3060)	1
  (10842, 2961)	1
  (10842, 3439)	1
  (10842, 3825)	1
  (10842, 96)	1
  (10842, 4170)	1
  (10842, 340)	1
  (10842, 1815)	1
  (10842, 3440)	1
  (10842, 3667)	1
  (10842, 3030)	1
  (10842, 3669)	1
  (10842, 278)	1
  (10842, 4162)	1
  (10842, 343)	1
  (10842, 758)	1
  (10842, 205)	1
  (10842, 416)	2
  (10842, 3392)	1
  (10842, 1997)	1


In [24]:
svm = SVC()
svm.fit(x_train,y_train)

SVC()

In [25]:
x_test = vectorizer.transform(test_dataset['DESCRIPTION'])
print(x_test.shape)
y_pred = svm.predict(x_test)


(54200, 5000)


In [26]:
print(len(y_pred))
pred = list()
for i in y_pred:
    pred.append(num_dict[i])
print(len(pred))

54200
54200


In [27]:
sol_dataset =  pd.read_csv(r"/content/sample_data/test_data_solution.txt",delimiter=':::',names=['ID','TITLE','GENRE','DESCRIPTION'],engine='python')
y_test = list(sol_dataset['GENRE'])
print(accuracy_score(pred,y_test))

0.5447416974169742
